<a href="https://colab.research.google.com/github/Sharma-Kamlesh/EVA5-Phase1/blob/main/Kamlesh_EVA5B2_assignment3X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms

In [2]:
#Model 

class Network(nn.Module):

  def __init__(self):
    super(Network,self).__init__()
    self.conv1  = nn.Conv2d(1,10, 3, padding=1)   
    self.conv2  = nn.Conv2d(10,10, 3, padding=1)
    self.pool1  = nn.MaxPool2d(2,2)  
    self.conv3  = nn.Conv2d(10,20, 3, padding=1)  
    self.conv4  = nn.Conv2d(20,20, 3, padding=1)
    self.pool2  = nn.MaxPool2d(2,2)     
    self.conv5  = nn.Conv2d(20,30, 3, padding=1)  
    self.conv6  = nn.Conv2d(30,62, 3)  
    self.GAP  = nn.AvgPool2d(5)

  def forward(self,t):
      #conv1
      t = self.conv1(t)     #28*28*1 -> 3*3*1*10 -> 28*28*10  RF:5
      t = F.relu(t)
      #conv2
      t = self.conv2(t)     #28*28*10 -> 3*3*1*10 -> 28*28*10  RF:7
      t = F.relu(t)
      t = self.pool1(t)     #28*28*10 ->  14*14*10  RF:14
      #conv3
      t = self.conv3(t)     #14*14*10 -> 3*3*10*20 -> 14*14*20  RF:16
      t = F.relu(t)
       #conv4
      t = self.conv4(t)     #14*14*20 -> 3*3*20*20 -> 14*14*20  RF:18
      t = F.relu(t)
      t = self.pool2(t)     #14*14*20 ->  7*7*20 RF:20
      #conv5
      t = self.conv5(t)     #7*7*20 -> 3*3*20*30 -> 7*7*30
      t = F.relu(t)
      #conv6
      t = self.conv6(t)     #7*7*30 ->  3*3*30*62 ->5*5*62
      t = F.relu(t)
      #output
      t = self.GAP(t)       #5*5*62 -> 1*1*62
    
     

      return F.log_softmax(t.view(-1,62),dim=1)

In [3]:
!pip install torchsummary                               #summurizes the give npytorch model , layer names, I/O shapes, kernal shape 
from torchsummary import summary
use_cuda = torch.cuda.is_available()                    # is_available() determines if the Cuda is available and initialises 
device = torch.device("cuda" if use_cuda else "cpu")    #if cuda tensor types is true , then it uses the GPu else CPU
model = Network().to(device)                                #
summary(model, input_size=(1, 28, 28))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 10, 28, 28]             910
         MaxPool2d-3           [-1, 10, 14, 14]               0
            Conv2d-4           [-1, 20, 14, 14]           1,820
            Conv2d-5           [-1, 20, 14, 14]           3,620
         MaxPool2d-6             [-1, 20, 7, 7]               0
            Conv2d-7             [-1, 30, 7, 7]           5,430
            Conv2d-8             [-1, 62, 5, 5]          16,802
         AvgPool2d-9             [-1, 62, 1, 1]               0
Total params: 28,682
Trainable params: 28,682
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.23
Params size (MB): 0.11
Estimated Total Size (MB): 0.34
---------------------------------------------

In [4]:
#download EMnist

train_set = torchvision.datasets.EMNIST(
    root='./data'
    ,split='byclass'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [5]:
#Batching the data

train_loader  = torch.utils.data.DataLoader(
    train_set,
    batch_size=512,
    shuffle=True
)

In [6]:
sample = next(iter(train_set))
image,  label = sample
image.shape,  image.unsqueeze(0).shape

(torch.Size([1, 28, 28]), torch.Size([1, 1, 28, 28]))

In [7]:
network = Network()
pred = network(image.unsqueeze(0))

In [8]:
pred.shape

torch.Size([1, 62])

In [9]:
label

35

In [10]:
pred.argmax(dim=1)

tensor([57])

In [11]:
data_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size =  512
)
batch = next(iter(data_loader))
len(batch)

2

In [12]:
images, labels  = batch

In [13]:
images.shape, labels.shape

(torch.Size([512, 1, 28, 28]), torch.Size([512]))

In [14]:
preds  = network(images)
preds.shape

torch.Size([512, 62])

In [15]:
def get_num_correct(preds, labels):
  return preds.squeeze().argmax(dim=1).eq(labels).sum().item()

In [16]:
get_num_correct(preds, labels)

3

In [ ]:

optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(20):

    total_loss = 0
    total_correct = 0

    for batch in data_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 350510 loss: 3051.8651689291
epoch 1 total_correct: 358554 loss: 2926.2961962223053
epoch 2 total_correct: 359634 loss: 2904.1204985380173
epoch 3 total_correct: 360119 loss: 2890.7994391918182
epoch 4 total_correct: 360295 loss: 2884.0110533237457
epoch 5 total_correct: 360534 loss: 2879.7714376449585
epoch 6 total_correct: 360685 loss: 2875.7225407361984
epoch 7 total_correct: 360839 loss: 2871.323142051697
epoch 8 total_correct: 360915 loss: 2869.3518291711807
